In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [ ]:
num_classes = 2

# Image Data Generators for augmenting and rescaling images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
# Load training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG16 expects 224x224 images
    batch_size=32,
    class_mode='categorical'  # Categorical for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [5]:
# Load the VGG16 model, excluding the top fully connected layers
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [6]:
# Freeze the VGG16 layers so they are not updated during training
base_model.trainable = False

In [7]:
# Build the additional layers on top of the base model
x = base_model.output

In [8]:
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Use Global Average Pooling to reduce dimensionality before the dense layers
x = GlobalAveragePooling2D()(x)

# Add fully connected layers with Dropout for regularization
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

In [9]:
# Output layer with softmax activation for multi-class classification
predictions = Dense(num_classes, activation='softmax')(x)

In [10]:
# Create the final model
model = Model(
    inputs = base_model.input,
    outputs = predictions
)

In [11]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate = 0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
# Display the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 3, 3, 512)      │             

 Total params: 20,620,354 (78.66 MB)

 Trainable params: 5,903,618 (22.52 MB)

 Non-trainable params: 14,716,736 (56.14 MB)

In [13]:
number_of_epochs = 30

In [14]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = number_of_epochs  # Adjust the number of epochs based on your dataset size
)

Epoch 1/30


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.5234 - loss: 1.0149 - val_accuracy: 0.4286 - val_loss: 0.7271
Epoch 2/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.0000e+00 - loss: 3.1232

2024-11-05 22:17:21.649181: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/envs/Thesis/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 523ms/step - accuracy: 0.0000e+00 - loss: 3.1232 - val_accuracy: 0.4286 - val_loss: 0.7341
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5572 - loss: 0.9161 - val_accuracy: 0.4286 - val_loss: 0.7190
Epoch 4/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7500 - loss: 0.6272

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 523ms/step - accuracy: 0.7500 - loss: 0.6272 - val_accuracy: 0.4286 - val_loss: 0.7153
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5732 - loss: 0.8424 - val_accuracy: 0.3929 - val_loss: 0.7014
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 523ms/step - accuracy: 0.6250 - loss: 0.6077 - val_accuracy: 0.3929 - val_loss: 0.6996
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.6201 - loss: 0.8017 - val_accuracy: 0.5357 - val_loss: 0.6933
Epoch 8/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6250 - loss: 0.8247

2024-11-05 22:18:28.045258: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 525ms/step - accuracy: 0.6250 - loss: 0.8247 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5991 - loss: 0.7938 - val_accuracy: 0.5714 - val_loss: 0.6896
Epoch 10/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 537ms/step - accuracy: 0.4688 - loss: 1.1133 - val_accuracy: 0.5714 - val_loss: 0.6890
Epoch 11/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.6659 - loss: 0.6669 - val_accuracy: 0.4643 - val_loss: 0.6875
Epoch 12/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 532ms/step - accuracy: 0.6875 - loss: 0.8441 - val_accuracy: 0.5000 - val_loss: 0.6872
Epoch 13/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6928 - loss: 0.6566 - val_accuracy: 0.4286 - val_loss: 0.6853
Epoch 14/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 508ms/step - accuracy: 0.6250 - loss: 0.8010 - val_accuracy: 0.4643 - val_loss: 0.6846
Epoch 15/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6870 - loss: 0.6137 - val_accuracy: 0.5714 - val_loss: 0.6834
Epoch

2024-11-05 22:19:55.881019: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 504ms/step - accuracy: 0.6875 - loss: 0.7347 - val_accuracy: 0.5714 - val_loss: 0.6833
Epoch 17/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.7149 - loss: 0.5467 - val_accuracy: 0.5000 - val_loss: 0.6825
Epoch 18/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 520ms/step - accuracy: 0.6875 - loss: 0.6606 - val_accuracy: 0.5000 - val_loss: 0.6831
Epoch 19/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6344 - loss: 0.6469 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 20/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 554ms/step - accuracy: 0.7500 - loss: 0.5047 - val_accuracy: 0.5000 - val_loss: 0.6978
Epoch 21/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.7920 - loss: 0.4463 - val_accuracy: 0.4643 - val_loss: 0.7260
Epoch 22/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 552ms/step - accuracy: 0.7188 - loss: 0.7833 - val_accuracy: 0.4643 - val_loss: 0.7269
Epoch 23/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.7054 - loss: 0.5999 - val_accuracy: 0.4643 - val_loss: 0.7151
Epoc

In [ ]:
# Optionally, unfreeze some layers in the base model and fine-tune
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = 10  # Fine-tune for a few more epochs
)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 42s 6s/step - accuracy: 0.7864 - loss: 0.5143 - val_accuracy: 0.5357 - val_loss: 0.6756
Epoch 2/5
1/6 ━━━━━━━━━━━━━━━━━━━━ 31s 6s/step - accuracy: 0.8438 - loss: 0.3157

2024-11-05 22:23:25.468071: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 495ms/step - accuracy: 0.8438 - loss: 0.3157 - val_accuracy: 0.5000 - val_loss: 0.6628
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.7409 - loss: 0.6558 - val_accuracy: 0.7500 - val_loss: 0.5845
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 499ms/step - accuracy: 0.8438 - loss: 0.4349 - val_accuracy: 0.7500 - val_loss: 0.5797
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 38s 6s/step - accuracy: 0.8166 - loss: 0.4416 - val_accuracy: 0.7143 - val_loss: 0.5767


In [25]:
# Save the model, optional
model.save(f'Pth_Files/VGG16_{number_of_epochs}.keras')

## Model Testing

In [46]:
def load_image_test(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array /= 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)
    predicted_probability = np.max(predictions) * 100  # Get the probability of the predicted class


    # If you know the class labels (e.g., from a class_indices mapping when using a data generator)
    class_labels = {0: 'Class1', 1: 'Class2', 2: 'Class3'}  # Update with your actual labels
    predicted_label = class_labels[predicted_class_index[0]]

    # Print the prediction results
    print(f"Predicted class index: {predicted_class_index[0]}")
    print(f"Predicted class label: {predicted_label}")
    print(f"Prediction confidence: {predicted_probability:.2f}%")

In [47]:
# Load your trained model
model = load_model(f'Pth_Files/VGG16_{number_of_epochs}.keras')

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 44 variables whereas the saved optimizer has 86 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [48]:
test_img_path = test_dir + '/Agaricus_abruptibulbus/Agaricus_abruptibulbus_1.jpg'
img_array = load_image_test(test_img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
Predicted class index: 0
Predicted class label: Class1
Prediction confidence: 56.19%


In [49]:
test_img_path = test_dir + '/Amanita_excelsa/Amanita_excelsa_1.jpg'
img_array = load_image_test(test_img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Predicted class index: 1
Predicted class label: Class2
Prediction confidence: 68.76%
